In [33]:
from fastcore.all import parallel
import pandas as pd
import requests 
import bs4
import tqdm
from pathlib import Path
import os
import re

get water user's water extraction place metadata

In [34]:
path = Path(".")

names = [Path(d.path).name for d in os.scandir(path/'permits/')]

In [45]:
def parse_html(name):
    html = open(f'permits/{name}').read()
    soup = bs4.BeautifulSoup(html)
    water_id = [{'name': name, 'water_extraction_place_id':d['id'].split('_')[-1]} for d in soup.findAll('div', id=re.compile('water_use_place'))]
    return water_id

In [46]:
results = parallel(parse_html, names, n_workers=8)



In [49]:
water_extraction_place_id = pd.DataFrame([dd for d in results for dd in d  ])

water_extraction_place_id['permit_id'] = water_extraction_place_id.name.str.split('.').str[0]

water_extraction_place_id.to_csv('water_extraction_place_id.csv', index=False)

get  coordinates

In [52]:
path = Path(".")

names = [Path(d.path) for d in os.scandir(path/'water_extract_locations')]

In [73]:
#export
def parse_water_extraction_file(name):
    """
    Function to parse water_extract_locations files and get information about location and type 
    of water extractions
    
    """
    soup = bs4.BeautifulSoup(open(name).read())
    
    user, idx_of_location = name.stem.split("_")
    
    descr = soup.select("div.form-group textarea")[0].text
    lat = soup.select('input#permitwaterplace-map_mark_y')[0]['value']
    lng = soup.select('input#permitwaterplace-map_mark_x')[0]['value']
    
    
    return {
        'user_name': user,
        'idx_of_location': idx_of_location,
        'description': descr,
        'lat': lat,
        'lng': lng,
    }
    

In [74]:
results = parallel(parse_water_extraction_file, names, n_workers=8)